# BLM442 Büyük Veri Final Projesi Kısım 2
* 170201088 - M. M. Enes YURTSEVER


* Veri seti içerisinde bir şirketin Ocak 2013 - Ekim 2015 tarihleri arasında gerçekleştirdiği günlük ürün satışı sayısını içermektedir.
* İlgili zaman serisindeki (time-series) sağlanan ürün, mağaza, eşya ve günlük satış bilgilerini kullanarak 2015 Kasım ayında eşyalar için **aylık satış sayısının** tahmin edilmesi beklenmektedir.
* Satış sayısı bilgileri günlük olarak verilmiştir. Aylık tahmin istendiği için ilgili train ve test veri seti içerisinde birtakım veri ön işleme teknikleri icra edilmelidir.
* Kasım 2015 için yapılan tahmin işlemlerinin **ID** ve **item_count_monthly** adında iki sütundan oluşan bir *submission.csv* dosyası içerisine kaydedilerek submit edilmesi istenmiştir.


## 1. Veri Ön İşleme Aşamaları, Sağlanan CSV Dosyalarının Okunması

In [ ]:
import numpy as np
import pandas as pd

from sklearn import *
import nltk, datetime
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
submission = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
item_cats = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')

train.head()
train.shape

In [ ]:
test.shape

### 1.1. Günlük satış verilerinin parse edilmesi
> Train veri setinde 'date' sütunu bulunur. Burada satış günü bilgisi, d.m.yyyy formatında verilmiştir. <br>
> Prediction işlemi aylık olarak yapılması beklendiğinden, ilgili sütun datetime formatına çevrilerek içerisinden ay ve yıl bilgisi çıkarılmıştır.


In [ ]:
# date sutununu, datetime formatina cevir.
# daha sonra date sutunundan month alinacak.
train['date'] = pd.to_datetime(train['date'], format = '%d.%m.%Y')
# date time'den aylari al.
train['month'] = train['date'].dt.month
# date time'den yillari al.
train['year'] = train['date'].dt.year

# aylik tahmin yapilacagi icin, ilgili df'lere parse edildi. daha sonra bu sutun, drop edilir.
train = train.drop(['date', 'item_price'], axis = 1)

### 1.2. Günlük satış adetlerinin gruplanarak aylık satış adetlerinin elde edilmesi
> Burada da train verisini, her bir eşya için günlük satış adetlerini toplayarak aylık olarak gruplandırıyoruz.<br>
Veri setinde halihazırda item_cnt_day olarak isimlendirilmiş günlük satış adetleri, her bir eşya için aylık satış adedini belirten gruplandırma neticesinde item_count_monthly olarak değiştirilmiştir.

In [ ]:
# her bir item'a ait gunluk satislari gruplayarak aylık satis sayilarini elde ediyoruz.
train = train.groupby([column for column in train.columns if column not in ['item_cnt_day']], as_index=False)[['item_cnt_day']].sum()
# gunluk satisi yapilan item sayaci, aya gore gruplandigindan sutun ismi degistirildi.
train = train.rename(columns={'item_cnt_day' : 'item_count_monthly'})


In [ ]:
train.head(10)

### 1.3. Train verisi için ekstra bir feature: Aylık ortalama satış adedi
>Uygulama içerisinde prediction işlemi için **Random Forest (RF) algoritması** kullanılacaktır.<br>
RF algoritmasında Big Data dersinde kullanılan daha önceki modellerden farklı olarak, **feature selection işlemi yapılmaz.**<br>
RF algoritmasına tüm sütunlar verilir. RF, bu feature sütunlarından rastgele feature seçimi yapar.<br>
Bu hususta prediction kalitesini artırmak amacıyla, her bir eşya için bulunan aylık ortalama satış adedi sütununun ortalamasını alarak **item_count_monthly_mean** adında bir feature sütunu oluşturuyoruz.

In [ ]:
# itemlerin aylik satis adetlerine gore ortalamasinin alinmasi
shop_item_ort = train[['shop_id', 'item_id', 'item_count_monthly']].groupby(['shop_id', 'item_id'], as_index=False)[['item_count_monthly']].mean()
#aylik satıs sayaci sutunu, aylik ortalama satis adedi olacak sekilde degisir (sutun adi)
shop_item_ort = shop_item_ort.rename(columns={'item_count_monthly' : 'item_count_monthly_mean'})

In [ ]:
# Elde ettigimiz ortalama ozelligini (shop_item_ort), train verimize ekliyoruz.
train = pd.merge(train, shop_item_ort, how='left', on=['shop_id', 'item_id'])

In [ ]:
# goruldugu uzere, olusturdugumuz item_count_monthly_mean sutunu, train verimize eklendi.
train.head(9)

In [ ]:
# veri setindeki en son ay icerisinde satisi yapilan urunleri aliyoruz. shop_son_ay = shop_prev_month
# 2013-2015 arasi satis bilgileri mevcuttur. 2013 ocak'ta date_block_num -> 0'dır. en son 2015 Ekim icin ilgili deger 33'tur.
shop_son_ay = train[train['date_block_num'] == 33][['shop_id', 'item_id', 'item_count_monthly']]
shop_son_ay = shop_son_ay.rename(columns={'item_count_monthly' : 'item_count_son_ay'})
shop_son_ay.head()

### 1.4. Train verisine diğer CSV'lerdeki tüm özelliklerin eklenmesi
> Daha önce de bahsedildiği gibi, RF algoritmasında feature selection işlemi yapılmamaktadır.<br>
Aşağıda verilen kodlarda, 2015 Ekim ayındaki her bir eşya için satış adedi, itemleri tutan dataframe, item kategorileri ve mağazalar, kısacası tüm featureler, train verisine eklenmektedir.<br>
RF, tüm bu özellikleri kullanarak bir model oluşturacaktır.

In [ ]:
# elde edilen son ay satis adetlerini de train verimize ekliyoruz.
train = pd.merge(train, shop_son_ay, how='left', on=['shop_id', 'item_id']).fillna(0.)
# train verisine item ozelliklerini ekliyoruz.
train = pd.merge(train, items, how='left', on='item_id')
# train verisine item kategorilerini ekliyoruz.
train = pd.merge(train, item_cats, how='left', on='item_category_id')
# train verisine dukkanlari (shop) ekliyoruz.
train = pd.merge(train, shops, how='left', on='shop_id')

# random forest ile predict yapilacagindan, train verisine tum featureler eklenmektedir.
# RF, tum bu featurelerden random secim yaparak egitimi gerceklestirecek
train.head()

### 1.5. Test verisinin ayarlanması
>Test verisinde öncelikle 2015 Kasım için prediction yapmak istediğimiz için, test verisinde ay, yıl ve ayı işaret eden date_block_num değişkenlerini, kasım için ayarlıyoruz.<br>
>Train verisine yapılan tüm veri ön işleme aşamaları test verisine de yapılır.<br>
>İlgili feature sütunları, train verisine de eklenir.<br>
>Son olarak test verisinde item_count_monthly sütunu, prediciton yapılacak sütundur. Burada sütunu init. etmek adına tüm satırlara 0 değeri atanır.

In [ ]:
# Test verisinin ayarlanmasi
# Uygulama icerisinde, 2015 Kasım'daki aylik satislari predict etmek istiyoruz.
# 2015 kasim bilgilerini ekliyoruz.
# train data'da 2015 ekimin date_block_number sayısı 33 oldugundan, 2015 kasim icin date_block_number'a 34 set ettik.
test['month'] = 11
test['year'] = 2015
test['date_block_num'] = 34
# ayrica RF ile train ve prediction islemleri yapilacagindan
# train verisine ekledigimiz tum ozellikleri test veri setine de ekleyecegiz (aylik ortalama satis, shop ve item bilgileri).

# Elde edilen ortalama ozelligini (shop_item_ort), test verimize ekliyoruz.
test = pd.merge(test, shop_item_ort, how='left', on=['shop_id', 'item_id']).fillna(0.)
# elde edilen son ay satis adetlerini de train verimize ekliyoruz.
test = pd.merge(test, shop_son_ay, how='left', on=['shop_id', 'item_id']).fillna(0.)
# test verisine item ozelliklerini ekliyoruz.
test = pd.merge(test, items, how='left', on='item_id')
# test verisine item kategorilerini ekliyoruz.
test = pd.merge(test, item_cats, how='left', on='item_category_id')
# test verisine dukkanlari (shop) ekliyoruz.
test = pd.merge(test, shops, how='left', on='shop_id')
# predict edilecek aylik satis sayisi sutunu degerlerini 0 olarak init. ediyoruz.
test['item_count_monthly'] = 0.

test.head()

In [ ]:
# RF Modeli kurulmadan once label encoding islemi
for column in ['shop_name', 'item_name', 'item_category_name']:
    label = preprocessing.LabelEncoder()
    label.fit(list(train[column].unique()) + list(test[column].unique()))
    train[column] = label.transform(train[column].astype(str))
    test[column] = label.transform(test[column].astype(str))
    print(column)

## 2. Random Forest Modelinin Kurulması ve RMSE Hesabı
>RF, bir ensemble modeldir. Çoklu öğrenme algoritmalarını kullanarak daha iyi bir prediction modeli oluşturmayı amaçlamaktadır.<br>
>RF ağaç yapısı için maksimum diyerarşik derinlik 25 olarak ayarlanmıştır.

In [ ]:
# Random Forest Alg. (RF) ile train ve prediction islemleri
column = [col for col in train.columns if col not in ['item_count_monthly']]
x1 = train[train['date_block_num'] < 33]
y1 = np.log1p(x1['item_count_monthly'].clip(0., 30.))
x1 = x1[column]

x2 = train[train['date_block_num'] == 33]
y2 = np.log1p(x2['item_count_monthly'].clip(0., 30.))
x2 = x2[column]

# RF bir ensemble modeldir. ensmble modeller, çoklu ogrenme algoritmalari kullanarak daha iyi prediction yapmayi amaclar.
# çoklu ogrenme alg. kullanarak daha iyi bir model egitmeyi amaclamaktadir.
rf_model = ensemble.ExtraTreesRegressor(n_estimators=25, n_jobs=-1, max_depth=25, random_state=18)
rf_model.fit(x1, y1)

print('RMSE degeri: ', np.sqrt(metrics.mean_squared_error(y2.clip(0., 30.), rf_model.predict(x2).clip(0., 30.))))



In [ ]:
print(test.columns.tolist())

### 2.1. Modelin train verisi için fit edilmesi ve tahmin işleminin tamamlanması
>RF modeli, veri setinin kırpılan bir bölümünde geliştirilir.<br>
>Prediction işleminden sonra submission.csv dosyası, ID ve item_count_monthly sütunları içerecek şekilde oluşturulur.

In [ ]:
# modeli train verisi icin fit edelim ve comp.ta istenen predictionlari hesaplayalim.
rf_model.fit(train[column], train['item_count_monthly'].clip(0., 30.))
# test verisinde daha oncesinde 0 olarak init ettigimiz kasım 2015 aylik satislari icin prediction islemi yapiyoruz.
test['item_count_monthly'] = rf_model.predict(test[column]).clip(0., 30.)
test['item_count_monthly'] = np.expm1(test['item_count_monthly'])

# predict edilen aylik satis adedini csv'ye yaziyoruz.
test[['ID', 'item_count_monthly']].to_csv('submission.csv', index=False)